In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt  
import seaborn as sns  
%matplotlib inline

In [29]:
c19 = pd.read_csv('../data/time_series_19-covid-Confirmed.csv')
c19a = c19[c19.columns[4:]]
case_sum = [list(c19a[i]) for i in c19a.columns]
c19b = c19.T[4:]
c19b['Confirmed'] = list(map(lambda x:sum(x), case_sum))
c19b = c19b.drop(columns=c19b.columns[:-1]).reset_index().rename(columns={'index':'Date'}).reset_index().rename(columns={'index':'Ticks'})
c19b['Date'] = pd.to_datetime(c19b['Date'])
X = c19b[['Date']]
y = np.array(c19b.Confirmed)
train_data = X[:10]
test_data = X[10:]


In [30]:
#First we’ll scale our train and test data with MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [33]:
#Before creating LSTM model we should create a Time Series Generator object.
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 12
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input)



ValueError: `start_index+length=12 > end_index=9` is disallowed, as no part of the sequence would be left to be used as current step.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

lstm_model = Sequential()
lstm_model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.summary()

lstm_model.fit_generator(generator,epochs=20)

losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize=(12,4))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,21,1))
plt.plot(range(len(losses_lstm)),losses_lstm);

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)



In [26]:
#As you know we scaled our data that’s why we have to inverse it to see true predictions.
lstm_predictions_scaled

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
lstm_predictions

test_data['LSTM_Predictions'] = lstm_predictions
test_data

test_data['Monthly beer production'].plot(figsize = (16,5), legend=True)
test_data['LSTM_Predictions'].plot(legend = True);

lstm_rmse_error = rmse(test_data['Monthly beer production'], test_data["LSTM_Predictions"])
lstm_mse_error = lstm_rmse_error**2
mean_value = df['Monthly beer production'].mean()

print(f'MSE Error: {lstm_mse_error}\nRMSE Error: {lstm_rmse_error}\nMean: {mean_value}')


NameError: name 'lstm_predictions_scaled' is not defined